**Напишите простую программу на Python, которая рассчитывает и выводит на экран среднюю доходность, стандартное отклонение и Sharpe Ratio для набора исторических данных о ценах акций.**

In [ ]:

import pandas as pd
import numpy as np
import yfinance as yf

def mean_std_sharpe_(ticker, start_date, end_date, source, risk_free_rate=0.02):
    if source == 'yahoo':
        # Загрузка данных из Yahoo!
        data = yf.download(ticker, start_date, end_date)
    elif source == 'csv':
        # Загрузка данных из CSV
        data = pd.read_csv(f'data/{ticker}.csv', index_col='Date', parse_dates=True)
        data = data.loc[start_date:end_date]
    else:
        raise ValueError("Неправильный источник данных. Используйте 'yahoo' или 'csv'.")

    data = data.dropna()
    # Преобразование в одномерный массив
    prices = data['Close'].to_numpy().flatten()
    # Расчет доходностей
    returns = pd.Series(prices).pct_change().dropna()
    mean_return = np.mean(returns)
    std_dev = np.std(returns)
    sharpe_ratio = (mean_return - risk_free_rate) / std_dev

    print('-' * 40)
    print(f"Средняя доходность: {mean_return:.4f}")
    print('-' * 40)
    print(f"Стандартное отклонение: {std_dev:.4f}")
    print('-' * 40)
    print(f"Коэффициент Шарпа: {sharpe_ratio:.4f}")

# Пример использования
mean_std_sharpe_('AAPL', '2016-01-01', '2019-08-01', 'yahoo', risk_free_rate=0.02)


**Напишите скрипт, который загружает исторические данные по акциям и автоматически генерирует записи о том, по каким критериям трейдер должен удалить стратегии из пула (например, ограничивайте процентные убытки или время бездействия).**

In [ ]:

# Пример использования с подгрузкой исторических данных yahoo! На нём писала и проверяла функцию.
import yfinance as yf
import pandas as pd

def show_records(ticker, start_date, end_date, loss_rate_input, inactivity_time_input, source):
    if source == 'yahoo':
        # Загрузка данных из Yahoo!
        data = yf.download(ticker, start=start_date, end=end_date)
    elif source == 'csv':
        # Загрузка данных из CSV файла
        data = pd.read_csv(f'data/{ticker}.csv', index_col='Date', parse_dates=True)
        data = data.loc[start_date:end_date]
    else:
        raise ValueError("Неправильный источник данных. Используйте 'yahoo' или 'csv'")

    # Расчет потерь и времени бездействия
    loss_rate = (data['Close'] - data['Open']).abs() / data['Open']
    inactivity_time = (data['Close'] - data['Open']).abs() / data['Close']

    criteria = {'loss_rate': loss_rate_input, 'inactivity_time': inactivity_time_input}
    records = []

    for i in range(len(data)):
        loss = loss_rate.iloc[i].item()
        inactivity = inactivity_time.iloc[i].item()

        if loss > criteria['loss_rate'] or inactivity > criteria['inactivity_time']:
            records.append({'Date': data.index[i], 'criteria': 'Удалить стратегию'})

    return records

if __name__ == "__main__":
    ticker = 'AAPL'
    start_date = '2016-01-01'
    end_date = '2019-08-01'
    loss_rate_input = 0.01
    inactivity_time_input = 30
    source = 'yahoo'
    records = show_records(ticker, start_date, end_date, loss_rate_input, inactivity_time_input, source)

    for record in records:
        print(record)
